In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Mounting the google drive

In [0]:
# Installing needed libraries

# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
!wget -q https://github.cFACE_RECOGNITION_Questionsom/creotiv/computer_vision/raw/master/resources.tar.gz -O resources.tar.gz
!tar xzf resources.tar.gz
!ls -la

    100% |████████████████████████████████| 25.4MB 1.9MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now
total 20
drwxr-xr-x 1 root root 4096 Mar 25 05:23 .
drwxr-xr-x 1 root root 4096 Mar 25 05:10 ..
drwxr-xr-x 1 root root 4096 Mar  8 17:25 .config
drwx------ 3 root root 4096 Mar 25 05:23 gdrive
-rw-r--r-- 1 root root    0 Mar 25 05:23 resources.tar.gz
drwxr-xr-x 1 root root 4096 Mar  8 17:26 sample_data


In [0]:
pip install https://github.com/imageio/imageio/archive/master.zip

     / 1.3MB 120.9MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-b6m9_vpc/wheels/ee/6c/aa/0b3af495a968d0d5de6143f36130915d7a5645b4c1a81e0b47
Successfully built imageio
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [0]:
import numpy as np
import random
import os
import imageio
import matplotlib.pyplot as plt
import pandas as pd
from time import time

import tensorflow as tf
tf.set_random_seed(1)

from keras.models import Model
from keras.layers import Input, Lambda, concatenate
from keras.optimizers import Adam
from keras import backend as K
from keras.layers import Conv2D, PReLU, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3

In [0]:
def get_model():
    no_top_model = InceptionV3(include_top=False, weights='imagenet', pooling='avg')

    x = no_top_model.output
    x = Dense(512, activation='elu', name='fc1')(x)
    x = Dense(128, name='fc2')(x)
    x = Lambda(lambda x: K.l2_normalize(x, axis=1), name='l2_norm')(x)
    return Model(no_top_model.inputs, x)

In [0]:
ALPHA = 0.2  # Triplet Loss Parameter

In [0]:
def get_triplets(features):
  df_features = pd.DataFrame(features)

  triplets = []
  for index, row in df_features.iterrows():
    same_tag = df_features.loc[df_features.iloc[:, -1] == row.iloc[-1]]
    same_tag_indexes = list(set(same_tag.index) - {index})
    diff_tag_indexes = list(set(df_features.index) - set(same_tag_indexes) - {index})

    anchor = row.iloc[0]
    anchor = anchor.reshape(-1, anchor.shape[0], anchor.shape[1], anchor.shape[2])
    
    pos = df_features.iloc[random.choice(same_tag_indexes), :].iloc[0]
    pos = pos.reshape(-1, pos.shape[0], pos.shape[1], pos.shape[2])
    
    neg = df_features.iloc[random.choice(diff_tag_indexes), :].iloc[0]
    neg = neg.reshape(-1, neg.shape[0], neg.shape[1], neg.shape[2])
  
    triplets.append(list(list([anchor, pos, neg])))
    
  return np.array(triplets)

In [0]:
def triplet_loss(x):
    anchor, positive, negative = tf.split(x, 3)

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), 1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), 1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), ALPHA)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), 0)

    return loss

In [0]:
#start_time = time()
numOfPhotosPerTag = 10
#Change this line to your own drive path
baseDir = "/content/gdrive/My Drive/images_crop/"
imagesHashtags = ["Ariel_Sharon", "Arnold_Schwarzenegger","Colin_Powell","Donald_Rumsfeld","George_W_Bush",
                 "Gerhard_Schroeder","Hugo_Chavez","Jacques_Chirac","Tony_Blair","Vladimir_Putin",
                 "Akshay_Kumar","Nawazuddin_Siddiqui","Salman_Khan","Shahrukh_Khan","Sunil_Shetty","Sunny_Deol"]
imagesDir = [baseDir + str(x) for x in imagesHashtags]
images = ["/" + str(x) + ".jpg" for x in range(1, numOfPhotosPerTag + 1)]
allImages = []

for x in imagesDir:
  allImages += [x + loc for loc in images]

In [0]:
imageio.imread(allImages[0], pilmode="RGB").shape
import cv2
data = []
for x in allImages:
  image = imageio.imread(x, pilmode="RGB")
  image = cv2.resize(image,(96,96))
  tag = x.split('/')[-2]
  data.append((image, tag))

In [0]:
triplet = get_triplets(data)


a = list([triplet[0][0],triplet[1][0],triplet[2][0],triplet[3][0],triplet[4][0],triplet[5][0],triplet[6][0],triplet[7][0],triplet[8][0],triplet[9][0],triplet[10][0],triplet[11][0],triplet[12][0],triplet[13][0],triplet[14][0],triplet[15][0]])
a = np.array(a)
a = np.squeeze(a)
p = list([triplet[0][1],triplet[1][1],triplet[2][1],triplet[3][1],triplet[4][1],triplet[5][1],triplet[6][1],triplet[7][1],triplet[8][1],triplet[9][1],triplet[10][1],triplet[11][1],triplet[12][1],triplet[13][1],triplet[14][1],triplet[15][1]])
p = np.array(p)
p = np.squeeze(p)
n = list([triplet[0][2],triplet[1][2],triplet[2][2],triplet[3][2],triplet[4][2],triplet[5][2],triplet[6][2],triplet[7][2],triplet[8][2],triplet[9][2],triplet[10][2],triplet[11][2],triplet[12][2],triplet[13][2],triplet[14][2],triplet[15][2]])
n = np.array(n)
n = np.squeeze(n)

In [0]:


nn4_small2 = get_model()


from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

# Input for anchor, positive and negative images
in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))


emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.8, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

In [0]:
nn4_small2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_75 (Activation)      (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
def triplet_generator(a,p,n):
    ''' Dummy triplet generator for API usage demo only.
    Will be replaced by a version that uses real image data later.
    :return: a batch of (anchor, positive, negative) triplets
    '''
    while True:
        #a_batch = vstack((triplets[1][0],triplets[2][0],triplets[3][0],triplets[4][0],triplets[5][0]))
        #p_batch = vstack((triplets[1][1],triplets[2][1],triplets[3][1],triplets[4][1],triplets[5][1]))
        
       # n_batch = vstack((triplets[1][2],triplets[2][2],triplets[3][2],triplets[4][2],triplets[5][2]))
        a_batch = a
        p_batch = p
        n_batch = n
       # a_batch = np.random.rand(4, 96, 96, 3)
       # p_batch = np.random.rand(4, 96, 96, 3)
       # n_batch = np.random.rand(4, 96, 96, 3)
        yield [a_batch , p_batch, n_batch], None
        

In [0]:
generator = triplet_generator(a,p,n) 

nn4_small2_train.compile(loss=None, optimizer='adam')

In [0]:




nn4_small2_train.fit_generator(generator, epochs=10,steps_per_epoch=5)

Epoch 1/10
5/5 [==============================] - 2s 306ms/step - loss: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 2s 309ms/step - loss: 0.0000e+00
Epoch 3/10
5/5 [==============================] - 2s 321ms/step - loss: 0.0000e+00
Epoch 4/10
5/5 [==============================] - 2s 316ms/step - loss: 0.0000e+00
Epoch 5/10
5/5 [==============================] - 2s 326ms/step - loss: 0.0000e+00
Epoch 6/10
5/5 [==============================] - 2s 328ms/step - loss: 0.0000e+00
Epoch 7/10
5/5 [==============================] - 2s 317ms/step - loss: 0.0000e+00
Epoch 8/10
5/5 [==============================] - 2s 318ms/step - loss: 0.0000e+00
Epoch 9/10
5/5 [==============================] - 2s 319ms/step - loss: 0.0000e+00
Epoch 10/10
5/5 [==============================] - 2s 321ms/step - loss: 0.0000e+00


In [0]:
d_a = nn4_small2.predict(np.expand_dims(np.squeeze(triplet[0][0]),axis=0))[0]

In [0]:
d_b = nn4_small2.predict(np.expand_dims(np.squeeze(triplet[0][1]),axis=0))[0]

In [0]:
dist1 = np.linalg.norm(d_a-d_b)

In [0]:
dist1

0.37002796

In [0]:
d_c = nn4_small2.predict(np.expand_dims(np.squeeze(triplet[2][2]),axis=0))[0]

In [0]:
dist2 = np.linalg.norm(d_a-d_c)

In [0]:
dist2

0.439299

In [0]:
from functools import partial, update_wrapper

# New Section